In [19]:
import scipy

In [4]:
# import numpy as np
#
# file_path = "offline_data/20M/mahjong-offline-data-batch-0.mat"
# raw_data = scipy.io.loadmat(file_path)
# action_array = np.reshape(raw_data['A'], -1)
# action_data = raw_data['A'].T
# obs_data = raw_data['X']
#
# mask = np.bitwise_and(action_array >= 34, action_array <= 45)
# mask = np.bitwise_and(mask, action_array != 41)
# mask = np.bitwise_and(mask, action_array != 42)
# mask = np.bitwise_and(mask, action_array != 43)
# obs_data_masked = obs_data[mask]  # 93 * 34
# action_data_masked = action_data[mask]  # cate: 9
#
# import torch
# action_data_masked_t = torch.tensor(action_data_masked)

TypeError: scatter() received an invalid combination of arguments - got (), but expected one of:
 * (int dim, Tensor index, Tensor src)
 * (int dim, Tensor index, Tensor src, *, str reduce)
 * (name dim, Tensor index, Tensor src)
 * (int dim, Tensor index, Number value)
 * (int dim, Tensor index, Number value, *, str reduce)
 * (name dim, Tensor index, Number value)


In [25]:
# print(action_data_masked_t.shape)
# class_labels = [34, 35, 36, 37, 38, 39, 40, 44, 45]
# new_action_data_masked_t = torch.tensor([class_labels.index(class_id.item()) for class_id in action_data_masked]).view(-1, 1)
#
# # new_action_data_masked_t.shape
# torch.zeros((95616, 9)).scatter_(1, new_action_data_masked_t.to(torch.int64), 1)

95616

In [21]:
from torch.utils.data import Dataset, DataLoader
import numpy as np
from torch import tensor
import torch


class MeldDataset(Dataset):
    def __init__(self, root_path='offline_data/20M', index=0, oracle=False):
        file_path = f"{root_path}/mahjong-offline-data-batch-{index}.mat"
        raw_data = scipy.io.loadmat(file_path)
        action_array = np.reshape(raw_data['A'], -1)
        action_data = raw_data['A'].T
        if oracle:
            obs_data = np.concatenate([raw_data['X'], raw_data['O']], axis=1)
        else:
            obs_data = raw_data['X']

        response_mask = np.bitwise_and(action_array >= 34, action_array <= 40)
        no_response_mask = action_array == 45
        # response 数据
        self.obs_data_masked = obs_data[response_mask]  # 93 * 34
        self.action_data_masked = action_data[response_mask]  # cate: 8
        # no response 数据
        no_response_sample = np.random.choice(np.arange(0, obs_data[no_response_mask].shape[0], 1), self.obs_data_masked.shape[0])
        no_response_obs = obs_data[no_response_mask][no_response_sample]
        no_response_action = action_data[no_response_mask][no_response_sample]
        # 连接两类数据
        self.obs_data_masked = np.concatenate([self.obs_data_masked, no_response_obs], axis=0)
        self.action_data_masked = np.concatenate([self.action_data_masked, no_response_action], axis=0)

        # onehot化
        action_category = [34, 35, 36, 37, 38, 39, 40, 45]
        self.action_data_masked = tensor([action_category.index(class_id.item()) for class_id in self.action_data_masked]).view(-1, 1)

        # noinspection PyTypeChecker
        self.action_data_masked_onehot = torch.zeros(self.action_data_masked.shape[0], 8).scatter_(1, self.action_data_masked, 1)

    def __len__(self):
        return len(self.action_data_masked)

    def __getitem__(self, idx):
        return tensor(self.obs_data_masked[idx], dtype=torch.float), self.action_data_masked_onehot[idx]

In [22]:
meld_dataset_valid = MeldDataset(index=39)

In [34]:
# for o, a in DataLoader(meld_dataset_valid, batch_size=10, shuffle=True):
#     print(o, a)
#     break

tensor([[[0., 1., 1.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 1., 1.,  ..., 0., 1., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 1., 1.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0., 

In [24]:
from torch import nn


class MeldModel(nn.Module):
    def __init__(self, in_channels=93):
        super().__init__()
        self.input_layer = nn.Sequential(nn.Conv1d(in_channels, 256, 3, padding=1),
                                         nn.ReLU())

        self.hidden_layer = nn.Sequential(nn.Conv1d(256, 256, 3, padding=1),
                                          nn.ReLU(),
                                          nn.Conv1d(256, 32, 3, padding=1),
                                          nn.ReLU(),
                                          nn.Flatten(),
                                          nn.Linear(32 * 34, 1024),
                                          nn.ReLU(),
                                          nn.Linear(1024, 256),
                                          nn.ReLU()
                                          )

        self.output_layer = nn.Sequential(nn.Linear(256, 8),
                                          nn.Softmax(dim=1))

    def forward(self, obs):
        return self.output_layer(self.hidden_layer(self.input_layer(obs)))

In [42]:
def train(model: nn.Module, train_dataset: Dataset, test_dataset: Dataset, loss_fn: nn.Module,
          optimizer: torch.optim.Optimizer,
          epoch: int = 1, train_batch_size=10, eval_interval=1000, device="cpu"):
    for e in range(epoch):
        dataloader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
        for iter_time, (obs, target_action) in enumerate(dataloader):
            model.train()

            obs = obs.to(device)
            target_action = target_action.to(device)
            model = model.to(device)

            action = model(obs)
            # print(action[0])
            # response_mask = np.where(target_action.argmax(axis=1).cpu() != 7, True, False)

            loss = loss_fn(action, target_action)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if iter_time % eval_interval == 0:
                evaluate(model, train_dataset, test_dataset, loss_fn, f"# epoch_{e}_iter_{iter_time} #", device=device)

In [35]:
def evaluate(model: nn.Module, train_dataset: Dataset, test_dataset: Dataset, loss_fn: nn.Module, log_title="", test_batch_size=10, device="cpu"):
    model.eval()
    model = model.to(device)
    train_dataloader = DataLoader(train_dataset, batch_size=test_batch_size, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=True)
    train_total_loss = []
    test_total_loss = []

    train_accuracy_total = 0
    train_accuracy = 0
    train_response_accuracy_total = 0
    train_response_accuracy = 0

    test_accuracy_total = 0
    test_accuracy = 0
    test_response_accuracy_total = 0
    test_response_accuracy = 0

    for obs, target_action in train_dataloader:
        obs = obs.to(device)
        target_action = target_action.to(device)

        action = model(obs)
        loss = loss_fn(action, target_action)
        train_total_loss.append(loss.item())

        action_argmax = action.argmax(axis=1)
        target_action_argmax = target_action.argmax(axis=1)

        train_accuracy_total += len(action_argmax)
        # noinspection PyTypeChecker
        train_accuracy += torch.sum(action_argmax == target_action_argmax).item()

        response_mask = np.where(target_action_argmax.cpu() != 7, True, False)
        train_response_accuracy_total += len(action_argmax[response_mask])
        # noinspection PyTypeChecker
        train_response_accuracy += torch.sum(action_argmax[response_mask] == target_action_argmax[response_mask]).item()

        # print(action_argmax[response_mask], target_action_argmax[response_mask])

    first_print = True
    for obs, target_action in test_dataloader:
        obs = obs.to(device)
        target_action = target_action.to(device)

        action = model(obs)
        loss = loss_fn(action, target_action)
        test_total_loss.append(loss.item())

        action_argmax = action.argmax(axis=1)
        target_action_argmax = target_action.argmax(axis=1)

        test_accuracy_total += len(action_argmax)
        # noinspection PyTypeChecker
        test_accuracy += torch.sum(action_argmax == target_action_argmax).item()

        response_mask = np.where(target_action_argmax.cpu() != 7, True, False)
        test_response_accuracy_total += len(action_argmax[response_mask])
        # noinspection PyTypeChecker
        test_response_accuracy += torch.sum(action_argmax[response_mask] == target_action_argmax[response_mask]).item()

        if first_print:
            first_print = False
            print(f"\nAction: {action_argmax} | Target Action: {target_action_argmax}\n")


    # noinspection all
    print(f"{log_title}\n"
          f"Train Loss: {np.mean(train_total_loss)} | Test Loss: {np.mean(test_total_loss)}\n"
          f"Train Accuracy: {train_accuracy / train_accuracy_total} | Test Accuracy: {test_accuracy / test_accuracy_total}\n"
          f"Train response accuracy: {train_response_accuracy / train_response_accuracy_total} | Test response accuracy: {test_response_accuracy / test_response_accuracy_total}\n"
          )

In [44]:
meld_model = MeldModel()

In [45]:
from torch.optim import Adam
adam_optimizer = Adam(meld_model.parameters(), lr=0.0001)

for i in range(39):
    train(meld_model, MeldDataset(index=i), meld_dataset_valid, nn.CrossEntropyLoss(), adam_optimizer, epoch=1, device="cuda")


Action: tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4], device='cuda:0') | Target Action: tensor([7, 7, 1, 3, 7, 7, 7, 3, 7, 7], device='cuda:0')

# epoch_0_iter_0 #
Train Loss: 2.079216684765286 | Test Loss: 2.0792186024475097
Train Accuracy: 0.012098310682077989 | Test Accuracy: 0.011779015427949555
Train response accuracy: 0.024196621364155978 | Test response accuracy: 0.02355803085589911


Action: tensor([7, 7, 7, 7, 7, 7, 7, 7, 7, 7], device='cuda:0') | Target Action: tensor([7, 7, 0, 7, 7, 7, 7, 7, 3, 7], device='cuda:0')

# epoch_0_iter_1000 #
Train Loss: 1.7737294650456261 | Test Loss: 1.7738528485488891
Train Accuracy: 0.5 | Test Accuracy: 0.5
Train response accuracy: 0.0 | Test response accuracy: 0.0


Action: tensor([7, 7, 3, 7, 7, 3, 7, 7, 7, 7], device='cuda:0') | Target Action: tensor([7, 0, 3, 7, 7, 3, 7, 2, 7, 3], device='cuda:0')

# epoch_0_iter_2000 #
Train Loss: 1.632731268519447 | Test Loss: 1.6287973349380493
Train Accuracy: 0.6389559253143655 | Test Accuracy: 0.6427245374

KeyboardInterrupt: 

In [46]:
evaluate(meld_model, MeldDataset(), meld_dataset_valid, nn.CrossEntropyLoss())


Action: tensor([7, 7, 7, 3, 3, 3, 7, 3, 7, 3]) | Target Action: tensor([7, 7, 7, 3, 3, 3, 0, 3, 7, 7])


Train Loss: 1.5658634155894082 | Test Loss: 1.5659889769363404
Train Accuracy: 0.7081481011050426 | Test Accuracy: 0.7078292042762947
Train response accuracy: 0.4958719674838054 | Test response accuracy: 0.4974073362780872



In [47]:
torch.save(meld_model.state_dict(), "weights/meld_model_1.566l_unfinished.pt")